In [ ]:
# Nombre: Joan Alonzo Machado Domínguez
# https://colab.research.google.com/drive/1KTqpB_1dycB0fNx6sBdd4RjOqEVHJ5kO?usp=sharing
# Actividad Guiada 3

In [ ]:
#!pip install request
import urllib.request

file = "swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f2fdbb160f0>)

In [ ]:
#!pip install tsplib95
import tsplib95 
import random
import math
from math import e, inf

problem = tsplib95.load(file)
Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())

In [ ]:
#se genera una solucion aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for i in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#devuelve la distancia entre dos nodos
def distancia(a, b, problem):
  return problem.get_weight(a, b)

#devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i], solucion[i+1], problem)

  return distancia_total + distancia(solucion[len(solucion) -1], solucion[0], problem)

s = crear_solucion(Nodos)
distancia_total(s, problem)

4424

In [ ]:
#busqueda aleatoria
def busqueda_aleatoria(problem, N):
  mejor_solucion = []
  mejor_distancia = inf

  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia = distancia_total(solucion, problem)

    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
  
  return mejor_solucion

s = busqueda_aleatoria(problem, 100)
print(distancia_total(s, problem), s)

4113 [0, 28, 7, 17, 13, 21, 20, 35, 32, 15, 31, 38, 6, 22, 10, 37, 1, 3, 25, 41, 39, 23, 12, 18, 11, 2, 26, 27, 5, 34, 4, 36, 14, 9, 40, 24, 29, 8, 30, 16, 33, 19]


In [ ]:
#busqueda vecina
def genera_vecina(solucion, problem):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  mejor_solucion = []
  mejor_distancia = 10e10

  for i in range(1, len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)

      if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
  
  return mejor_solucion

  s1 = crear_solucion(Nodos)
  s2 = genera_vecina(s1, problem)

  print(distancia_total(s1, problem), s1)
  print(distancia_total(s2, problem), s2)


In [ ]:
#busqueda local
def busqueda_local(problem):
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = 10e10

  iteracion=0
  while(1):
    iteracion+=1
    vecina = genera_vecina(solucion_referencia, problem)
    distancia_vecina = distancia_total(vecina, problem)

    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    else:
      print("En la iteracion "+ str(iteracion) +" encontramos la solucion " + str(mejor_distancia) )
      print(mejor_solucion)
      return mejor_solucion

    solucion_referencia = vecina

busqueda_local(problem)

En la iteracion 41 encontramos la solucion 1756
[0, 7, 14, 19, 13, 11, 12, 18, 26, 5, 16, 15, 6, 4, 10, 25, 41, 23, 39, 22, 38, 33, 35, 36, 37, 17, 31, 20, 34, 32, 27, 28, 30, 21, 24, 40, 9, 8, 29, 2, 3, 1]


[0,
 7,
 14,
 19,
 13,
 11,
 12,
 18,
 26,
 5,
 16,
 15,
 6,
 4,
 10,
 25,
 41,
 23,
 39,
 22,
 38,
 33,
 35,
 36,
 37,
 17,
 31,
 20,
 34,
 32,
 27,
 28,
 30,
 21,
 24,
 40,
 9,
 8,
 29,
 2,
 3,
 1]

In [ ]:
#genera vecina aleatoria
def genera_vecina_aleatoria(solucion):
  #generador de 1 solucion vecina 2-opt aleatoria (intercambiar 2 nodos)
  #se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample(range(1, len(solucion)), 2))
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

def bajar_temperatura(T):
  return T*0.99

def probabilidad(T,d):
  return random.random() < math.exp(-1*d / T)
  
probabilidad(0.0001, 100)


False

In [ ]:
import copy

def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []
  mejor_distancia = 10e100

  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solucion vecina
    vecina = genera_vecina_aleatoria(solucion_referencia)

    #calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)

    #si es la mejor solucion de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina

    #Si la nueva vecina es mejor se cambia
    #Si es peor se cambia segun una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = copy.deepcopy(vecina)
      distancia_referencia = distancia_vecina

    #bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solucion encontrada es ", end="")
  print(mejor_solucion)
  print("con una distancia total de ", end="")
  print(mejor_distancia)
  return mejor_solucion

sol = recocido_simulado(problem, 1000000)

La mejor solucion encontrada es [0, 28, 29, 8, 10, 41, 23, 40, 24, 21, 9, 6, 5, 15, 37, 17, 31, 33, 38, 22, 39, 30, 34, 14, 16, 19, 13, 26, 12, 25, 11, 18, 2, 27, 3, 4, 1, 7, 36, 35, 20, 32]
con una distancia total de 1967


##Algoritmo de colonia de hormigas

La función Add_Nodo selecciona al azar un nodo con probabilidad uniforme.
Para ser mas eficiente debería seleccionar el próximo nodo siguiendo la probabilidad correspondiente a la ecuación:

$p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}$, si $j \in J^k_i$

$p^k_{ij}(t) = 0$, si $j \notin J^k_i$

In [ ]:
#funciones auxiliares
def Add_Nodo(problem, H, T):
  #mejora: establecer una funcion de probabilidad para añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(list(set(range(1, len(Nodos))) - set(H) ))

def Incrementa_Feromona(problem, T, H):
  #incrementa segun la calidad de la solucion, añadir una cantidad inversamente proporcional a la distancia total
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #evapora 0.3 el valor de la feromona, sin que baje de 1
  #mejora: podemos elegir diferentes funciones de evaporacion dependiendo de la cantidad actual y de la suma total de feromonas depositadas...
  T = [[max(T[i][j] - 0.3, 1) for i in range(len(Nodos)) ] for j in range(len(Nodos)) ]
  return T

#funcion hormigas
def hormigas(problem, N):
  #problem = datos del problema
  #N = Numero de agentes(hormigas)

  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas 
  Aristas = list(problem.get_edges())

  #inicializa las aristas con una cantidad inicial de feromonas:1
  #mejora: inicializar con valores diferentes dependiento diferentes criterios
  T = [[1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]

  #se generan los agentes(hormigas) que seran estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]

  #recorre cada agente construyendo la solucion
  for h in range(N):
    #para cada agente se construye un camino
    for i in range(len(Nodos)-1):

      #elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h], T)
      Hormiga[h].append(Nuevo_Nodo)

    #incrementa feromonas en esa arista
    T = Incrementa_Feromona(problem, T, Hormiga[h])

    #Evapora Feromona
    T = Evaporar_Feromonas(T)

  #seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N):
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia = distancia_actual

  print(mejor_solucion)
  print(mejor_distancia)

hormigas(problem, 1000)

[0, 39, 9, 6, 17, 37, 4, 22, 34, 24, 3, 30, 7, 38, 33, 21, 27, 18, 16, 36, 15, 14, 13, 26, 31, 35, 5, 32, 23, 11, 12, 25, 10, 1, 2, 40, 41, 29, 20, 28, 8, 19]
3954


#### Actividad Individual

- Iterar, generar nuevas hormigas con la inforamción de las feromonas
- Función de probabilidad para elegir el siguiente nodo(parámetros α y β)